In [165]:
import pandas as pd
import numpy as np
from datetime import date
from datetime import datetime
from IPython.display import display, HTML

In [166]:
def selecionar_aprovados(dados, coluna_aprovado_discursiva, coluna_situacao):

    df = dados.copy(deep=True)

    # Seleciona os 81 primeiros candidatos aprovados
    df.loc[df.loc[df[coluna_aprovado_discursiva] == True].head(81).index, coluna_situacao] = "Aprovado nas vagas de ampla concorrência"

    # Seleciona os candidatos aprovados PCD que ainda não foram incluídos
    df.loc[df.loc[(df[coluna_situacao].isna()) & 
                  (df["SITUACAO OBJETIVA"] == "Aprovado PcD") &
                  (df[coluna_aprovado_discursiva] == True)].head(9).index, coluna_situacao] = "Aprovado nas vagas de cota PCD"

    # Caso não haja 9 candidatos aprovados PCD, completa com aprovados na Ampla
    if len(df[df[coluna_situacao] == "Aprovado nas vagas de cota PCD"]) < 9:
        df.loc[df.loc[(df[coluna_situacao].isna()) &
                      (df[coluna_aprovado_discursiva] == True)].head(9 - len(df[df[coluna_situacao] == "Aprovado nas vagas de cota PCD"])).index, coluna_situacao] = "Aprovado nas vagas de ampla concorrência"

    df.loc[(df[coluna_situacao].isna()) & 
           (df[coluna_aprovado_discursiva] == True), coluna_situacao] = "Cadastro de Reservas"

    df[coluna_situacao] = df[coluna_situacao].fillna('Reprovado na discursiva')
      
    # Retorna os 81 primeiros aprovados + os 9 primeiros aprovados PCD
    return df

In [167]:
def gerar_html_preliminar_linha(row):
    return f'''<tr style="background-color: rgba({'193, 225, 193' if (row['SITUAÇÃO PRELIMINAR'] == "Aprovado nas vagas de ampla concorrência") | (row['SITUAÇÃO PRELIMINAR'] == "Aprovado nas vagas de cota PCD") else
     '255, 250, 160' if row['SITUAÇÃO PRELIMINAR'] == "Cadastro de Reservas" else 
     '250, 160, 160'},{'1' if row.name % 2 == 0 else '0.8'});">
    <td>{int(row['POS_p']) if ~np.isnan(row['POS_p']) else "Eliminado"}</td>
    <td>{row['INSC_NUMB']}</td>
    <td>{row['NOME']}</td>
    <td>{row['TOTAL_p']}</td>
    <td>{'SIM' if row['IDOSO'] else 'NÃO'}</td>
    <td>{row['TOTAL DISCURSIVA_p'] if ~np.isnan(row['TOTAL DISCURSIVA_p']) else '-'}</td>
    <td>{row['P2']}</td>
    <td>{row['CD']}</td>
    <td>{row['DEV']}</td>
    <td>{row['BD']}</td>
    <td>{row['NASC'].strftime('%d/%m/%Y')}</td>
    <td>{'Eliminado' if np.isnan(row['POS_p']) else
         '<b>▲ </b>' + str(int(row['POS OBJETIVA'] - row['POS_p'])) if row['POS OBJETIVA'] > row['POS_p'] else 
         '-' if row['POS OBJETIVA'] == row['POS_p'] else
         '<b>▼ </b>' + str(int(row['POS_p'] - row['POS OBJETIVA']))}</td>
    <td>{row['SITUAÇÃO PRELIMINAR']}</td>
    </tr>'''

In [168]:
def gerar_html_definitivo_linha(row):
    return f'''<tr style="background-color: rgba({'193, 225, 193' if (row['SITUAÇÃO DEFINITIVA'] == "Aprovado nas vagas de ampla concorrência") | (row['SITUAÇÃO DEFINITIVA'] == "Aprovado nas vagas de cota PCD") else
     '255, 250, 160' if row['SITUAÇÃO DEFINITIVA'] == "Cadastro de Reservas" else 
     '250, 160, 160'},{'1' if row.name % 2 == 0 else '0.8'});">
    <td>{int(row['POS_d']) if ~np.isnan(row['POS_d']) else "Eliminado"}</td>
    <td>{row['INSC_NUMB']}</td>
    <td title="{row['NOME']}
          Língua Portuguesa: {row['PT']}/8
          Raciocínio Lógico-Matemático: {row['RLM']}/8
          Direito Administrativo e Legislação Específica:  {row['DALE']}/12
          Direito Constitucional e Direitos Humanos: {row['DCDH']}/8
          Direito Civil e Direito Empresarial: {row['DCDE']}/10
          Direito Penal: {row['DP']}/6
          Direito Tributário: {row['DT']}/10
          Contabilidade Geral: {row['CONT']}/10
          Economia e Matemática Financeira: {row['EMF']}/8
          Total P1: {row['P1']}/80
          
          Língua Inglesa: {row['ING']}/12
          Ciência de Dados: {row['CD']}/20
          Desenvolvimento de Sistemas: {row['DEV']}/16
          Banco de dados: {row['BD']}/12
          Infraestrutura Tecnológica: {row['IT']}/12
          Segurança da Informação: {row['SEG']}/12
          Legislação Tributária do Estado de Minas Gerais: {row['LTE']}/16
          Total P2: {row['P2']}/100

          Questão 1: {row['Q1_d'] if row['APROVADO DISCURSIVA_d'] else '-'}/20
          Questão 2: {row['Q2_d'] if row['APROVADO DISCURSIVA_d'] else '-'}/20
          Questão 3: {row['Q3_d'] if row['APROVADO DISCURSIVA_d'] else '-'}/20
          Questão 4: {row['Q4_d'] if row['APROVADO DISCURSIVA_d'] else '-'}/20
          Total P3: {row['TOTAL DISCURSIVA_d'] if row['APROVADO DISCURSIVA_d'] else '-'}/80

          Total: {row['TOTAL_d']}/260">{row['NOME']}</td>
    <td>{row['TOTAL_d']}</td>
    <td>{'SIM' if row['IDOSO'] else 'NÃO'}</td>
    <td>{row['TOTAL DISCURSIVA_d'] if ~np.isnan(row['TOTAL DISCURSIVA_d']) else '-'}</td>
    <td>{row['P2']}</td>
    <td>{row['CD']}</td>
    <td>{row['DEV']}</td>
    <td>{row['BD']}</td>
    <td>{row['NASC'].strftime('%d/%m/%Y')}</td>
    <td>{'Eliminado' if np.isnan(row['POS_d']) else
         '<b>▲ </b>' + str(int(row['POS OBJETIVA'] - row['POS_d'])) if row['POS OBJETIVA'] > row['POS_d'] else 
         '-' if row['POS OBJETIVA'] == row['POS_d'] else
         '<b>▼ </b>' + str(int(row['POS_d'] - row['POS OBJETIVA']))}</td>
    <td>{'Eliminado' if np.isnan(row['POS_d']) else
         '★' if np.isnan(row['POS_p']) else
         '<b>▲ </b>' + str(int(row['POS_p'] - row['POS_d'])) if row['POS_p'] > row['POS_d'] else 
         '-' if row['POS_p'] == row['POS_d'] else
         '<b>▼ </b>' + str(int(row['POS_d'] - row['POS_p']))}</td>
    <td>{row['SITUAÇÃO DEFINITIVA']}</td>
    </tr>'''

In [169]:
resultado_objetiva = pd.read_csv('./data/resultado-objetiva.csv', sep=";", index_col=['INSC_NUMB', 'NOME'])
resultado_objetiva['NASC'] = pd.to_datetime(resultado_objetiva['NASC'], format='%d/%m/%Y')
resultado_objetiva['IDADE'] = (datetime.today() - resultado_objetiva['NASC']).astype('timedelta64[Y]')
resultado_objetiva['IDOSO'] = resultado_objetiva['IDADE'] >= 60

resultado_discursiva_preliminar = pd.read_csv('./data/resultado-discursiva-preliminar.csv', sep=";", index_col=['INSC_NUMB', 'NOME'])
resultado_discursiva_definitivo = pd.read_csv('./data/resultado-discursiva-definitivo.csv', sep=";", index_col=['INSC_NUMB', 'NOME'])

In [170]:
resultado_definitivo = resultado_objetiva.merge(resultado_discursiva_preliminar, on=['INSC_NUMB', 'NOME'], how='left').merge(resultado_discursiva_definitivo, on=['INSC_NUMB', 'NOME'], how='left', suffixes=('_p', '_d'))

In [171]:
resultado_definitivo['TOTAL_p'] = resultado_definitivo['TOTAL OBJETIVA'] + resultado_definitivo['TOTAL DISCURSIVA_p'].fillna(value=0)
resultado_definitivo['APROVADO DISCURSIVA_p'] = resultado_definitivo['TOTAL DISCURSIVA_p'] >= 40

resultado_definitivo['TOTAL_d'] = resultado_definitivo['TOTAL OBJETIVA'] + resultado_definitivo['TOTAL DISCURSIVA_d'].fillna(value=0)
resultado_definitivo['APROVADO DISCURSIVA_d'] = resultado_definitivo['TOTAL DISCURSIVA_d'] >= 40


In [172]:
resultado_definitivo_ordenado = resultado_definitivo.sort_values(by=['APROVADO DISCURSIVA_p', 'TOTAL_p', 'IDOSO', 'TOTAL DISCURSIVA_d', 'P2', 'CD', 'DEV', 'BD', 'NASC'], 
                                               ascending=[False, False, False, False, False, False, False, False, True])
resultado_definitivo_ordenado = resultado_definitivo_ordenado.reset_index(drop=False)
resultado_definitivo_ordenado['POS_p'] = np.where(~np.isnan(resultado_definitivo_ordenado['TOTAL DISCURSIVA_p']), resultado_definitivo_ordenado.index + 1, np.nan)
resultado_definitivo_ordenado = selecionar_aprovados(resultado_definitivo_ordenado, "APROVADO DISCURSIVA_p", "SITUAÇÃO PRELIMINAR")
resultado_definitivo_ordenado['html_preliminar_linha'] = resultado_definitivo_ordenado.apply(gerar_html_preliminar_linha, axis=1)

In [173]:
render_html_preliminar = '''<style>
table {
  border-collapse: collapse;
  font-size: 12px;
}
table, th, td {
  border: 1px solid black;
  text-align: center;
}
th {
  font-weight: bold;
  top: 0;
}
body {
  font-family: Arial, sans-serif;
}
thead { 
  display: table-header-group; 
}
@media print {
  tr {
    page-break-inside: avoid;
  }
}
@page {
  size: A3;
}
</style>
<h4><center>Resultado Preliminar - Tecnologia da Informação<br>Secretaria de Estado de Fazenda de Minas Gerais</center></h4>
Critérios de desempate:<br>
a) tiver idade igual ou superior a sessenta anos, nos termos do art. 27, parágrafo único, do Estatuto do Idoso;<br>
b) obtiver maior nota na Prova Escrita Discursiva;<br>
c) obtiver maior nota na Prova Escrita Objetiva II;<br>
d) obtiver maior número de acertos em Ciência de Dados;<br>
e) obtiver maior número de acertos em Desenvolvimento de Sistemas;<br>
f) obtiver maior número de acertos em Banco de Dados;<br>
g) tiver exercido a função de jurado, nos termos do art. 440 do Código de Processo Penal (não considerado nesta classificação);<br>
h) persistindo o empate, observar-se-á o critério de maior idade.
<table style="margin-top: 10px;">
  <colgroup>
    <col style="width: 7%;">
    <col style="width: 7%;">
    <col style="width: 24%;">
    <col style="width: 5%;">
    <col style="width: 4%;">
    <col style="width: 4%;">
    <col style="width: 4%;">
    <col style="width: 4%;">
    <col style="width: 4%;">
    <col style="width: 4%;">
    <col style="width: 9%;">
    <col style="width: 8%;">
    <col style="width: 16%;">
  </colgroup>
  <thead>
<tr>
<th rowspan="2">Posição preliminar</th>
<th rowspan="2">Inscrição</th>
<th rowspan="2">Nome</th>
<th rowspan="2">Nota total preliminar</th>
<th colspan="7">Critérios de desempate</th>
<th rowspan="2">Variação de posição</th>
<th rowspan="2">Situação preliminar</th>
</tr>
<tr>
    <th>Idoso</th>
    <th>P3</th>
    <th>P2</th>
    <th>CD</th>
    <th>DS</th>
    <th>BD</th>
    <th>Data de Nascimento</th>
</tr>
</thead>'''

for index, row in resultado_definitivo_ordenado.iterrows():
    render_html_preliminar += row['html_preliminar_linha']

render_html_preliminar += '''</table>'''

In [174]:
resultado_definitivo_ordenado = resultado_definitivo_ordenado.sort_values(by=['APROVADO DISCURSIVA_d', 'TOTAL_d', 'IDOSO', 'TOTAL DISCURSIVA_d', 'P2', 'CD', 'DEV', 'BD', 'NASC'], 
                                               ascending=[False, False, False, False, False, False, False, False, True])
resultado_definitivo_ordenado = resultado_definitivo_ordenado.reset_index(drop=False)
resultado_definitivo_ordenado['POS_d'] = np.where(~np.isnan(resultado_definitivo_ordenado['TOTAL DISCURSIVA_d']), resultado_definitivo_ordenado.index + 1, np.nan)
resultado_definitivo_ordenado = selecionar_aprovados(resultado_definitivo_ordenado, "APROVADO DISCURSIVA_d", "SITUAÇÃO DEFINITIVA")
resultado_definitivo_ordenado['html_definitivo_linha'] = resultado_definitivo_ordenado.apply(gerar_html_definitivo_linha, axis=1)

In [175]:
render_html_definitivo = '''<style>
table {
  border-collapse: collapse;
  font-size: 12px;
}
table, th, td {
  border: 1px solid black;
  text-align: center;
}
th {
  font-weight: bold;
  top: 0;
}
body {
  font-family: Arial, sans-serif;
}
thead { 
  display: table-header-group; 
}
@media print {
  tr {
    page-break-inside: avoid;
  }
}
@page {
  size: A3;
}
</style>
<h4><center>Resultado Definitivo - Tecnologia da Informação<br>Secretaria de Estado de Fazenda de Minas Gerais</center></h4>
Critérios de desempate:<br>
a) tiver idade igual ou superior a sessenta anos, nos termos do art. 27, parágrafo único, do Estatuto do Idoso;<br>
b) obtiver maior nota na Prova Escrita Discursiva;<br>
c) obtiver maior nota na Prova Escrita Objetiva II;<br>
d) obtiver maior número de acertos em Ciência de Dados;<br>
e) obtiver maior número de acertos em Desenvolvimento de Sistemas;<br>
f) obtiver maior número de acertos em Banco de Dados;<br>
g) tiver exercido a função de jurado, nos termos do art. 440 do Código de Processo Penal (não considerado nesta classificação);<br>
h) persistindo o empate, observar-se-á o critério de maior idade.
<table style="margin-top: 10px;">
  <colgroup>
    <col style="width: 7%;">
    <col style="width: 7%;">
    <col style="width: 23%;">
    <col style="width: 5%;">
    <col style="width: 4%;">
    <col style="width: 4%;">
    <col style="width: 4%;">
    <col style="width: 4%;">
    <col style="width: 4%;">
    <col style="width: 4%;">
    <col style="width: 7%;">
    <col style="width: 7%;">
    <col style="width: 7%;">
    <col style="width: 13%;">
  </colgroup>
  <thead>
<tr>
<th rowspan="2">Posição preliminar</th>
<th rowspan="2">Inscrição</th>
<th rowspan="2">Nome</th>
<th rowspan="2">Nota total preliminar</th>
<th colspan="7">Critérios de desempate</th>
<th rowspan="2">Variação de posição em relação a prova objetiva</th>
<th rowspan="2">Variação de posição em relação ao resultado preliminar</th>
<th rowspan="2">Situação preliminar</th>
</tr>
<tr>
    <th>Idoso</th>
    <th>P3</th>
    <th>P2</th>
    <th>CD</th>
    <th>DS</th>
    <th>BD</th>
    <th>Data de Nascimento</th>
</tr>
</thead>'''

for index, row in resultado_definitivo_ordenado.iterrows():
    render_html_definitivo += row['html_definitivo_linha']

render_html_definitivo += '''</table>'''

In [176]:
with open('./output/resultado_preliminar.html', 'w') as f:
    f.write(render_html_preliminar)

with open('./output/resultado_definitivo.html', 'w') as f:
    f.write(render_html_definitivo)

In [177]:
display(HTML(render_html_preliminar))

In [178]:
display(HTML(render_html_definitivo))